# Deep Neural Network for MNIST Classification

In this project, I am to essentially build the 'Hello World!' of Deep Learning. I hope to apply all the knowledge I have gained in my courses to build my very first deep learning algorithm.

The dataset, MNIST, is for Handwritten Digit Recognition. This project aims to classify each handwritten digit to the actual digit. The goal is to build a deep neural network.

In [4]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds